In [1]:
from tqdm import tqdm
tqdm.pandas()
from glob import glob

import json
import csv
import numpy as np
import pandas as pd
from collections import Counter

import rdflib
from rdflib import Graph
from data.data import CollectionAccessor, ImageHandler, get_latest

from search import Search, Randomiser

/home/valentin/home2-env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:

# dmg_meta = dict(name="DMG 2025-04-03", id_="DMG_2025-04-03",
#                 creation_timestamp="2025-04-03")
# # dmg = CollectionAccessor.get_DMG("./data/DMG_2025-03-26.csv", "", **dmg_meta)
# image_folder = "./data/images/DMG"
# img = ImageHandler(image_folder=image_folder)

# dmg = CollectionAccessor.get_DMG(pub_path="./data/dumps/API_dump_public_2025-04-03_extracted.csv",
#                                  priv_path="./data/dumps/API_dump_private_2025-04-03_extracted.csv",
#                                  rights_path="./data/rights.csv",
#                                  image_handler=img,
#                                  **dmg_meta)

In [39]:
def init_DMG():
    image_folder = "./data/images/DMG"
    image_handler = ImageHandler(image_folder=image_folder)

    dmg_meta = dict(name="Design Museum Gent (public & private)", id_="DMG_2025-05-06",
                creation_timestamp="2025-05-06")
    df = CollectionAccessor.get_DMG(pub_path=get_latest("./data/dumps", contains="public"),
                                     priv_path=get_latest("./data/dumps", contains="private"),
                                     rights_path="./data/rights.csv",
                                     image_handler=image_handler,
                                     **dmg_meta)
    
    rand = Randomiser(df, name="R1")
    rand2 = Randomiser(df, name="R2")
    s = Search([rand, rand2])
    return df, s

dmg, dmg_search = init_DMG()

100%|█████████████████████████████| 24824/24824 [00:00<00:00, 524901.20it/s]


---
# DEV FUNCTIONS

In [40]:
def parse_id_list(id_list_str):
    try:
        return list(map(str.strip, id_list_str.split(",")))
    except ValueError:
        raise s

In [41]:
def search_collection(collection_id, object_ids, concept, model_list):
    cur_coll = get_collection(collection_id)
    object_ids = parse_id_list(object_ids)
    model_list = parse_id_list(model_list)
    cur_search = searches[collection_id]
    print(object_ids)
    scores = cur_search(object_ids)


object_ids = "1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"
object_ids = parse_id_list(object_ids)
object_ids
recs = dmg.loc[object_ids]

In [42]:
# searcher_scores = [cur_s(recs) for cur_s in dmg_search.searchers]
# searcher_scores = pd.DataFrame({cur_s.name: cur_s for cur_s in searcher_scores})
# searcher_scores.loc[recs.index] = 0.


# searcher_scores.round(3)

,R1,R2
object_number,,
1992-0004_0-2,0.716,0.848
1992-0004_1-2,0.255,0.244
1992-0004_2-2,0.202,0.023
4521,0.901,0.840
5051,0.388,0.050
...,...,...
2022-0028,0.184,0.891
2022-0025_0-3,0.841,0.669
2022-0025_1-3,0.658,0.908


---
# TESTING ROUTES

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/object-details?object_ids=1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/list-models"

In [ ]:
!curl "http://0.0.0.0:8080/moon?ISO_8601_datetime=2025-12-17&lat_degrees=51.05&long_degrees=3.71"

In [22]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b""

{"1992-0004_0-2":0.8271618818115816,"1992-0004_1-2":0.0565617658428772,"1992-0004_2-2":0.4564151015244897,"4521":0.9473070661187916,"5051":0.22006131900523584,"5052":0.7891887227434542,"5055":0.6235876183317619,"5057_1-2":0.5311479204324407,"5057_2-2":0.5720745645261427,"5056":0.026454104632222752,"0901":0.885286629879073,"0977":0.7804882596976119,"0980":0.322821535369918,"0008":0.0533544000252103,"0832":0.021974578161007563,"0833":0.15433167607512066,"0834":0.2644309949368254,"0836":0.6244541717829577,"0843":0.9051807505410048,"0844":0.8428667145372197,"0847":0.6754600155571419,"0848":0.2644955705063251,"0849":0.015398779493755455,"0850":0.43213152198941784,"0855":0.051308893344283346,"0856":0.8336913801494108,"0864":0.15905365931400828,"0867":0.7056619136418143,"0870":0.172245897557601,"0872":0.2413949173432447,"0873":0.23006043211928595,"0875":0.5906313412057703,"0876":0.4831088671822078,"0879":0.46994669529466215,"0880":0.17644362999900498,"0881":0.9856986804748514,"0892":0.1534799

In [53]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search/order?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&skip=200&limit=100&reverse=true"

[{"inventory_number":"TUP-0120","title":"Eleganzia Bowl & Seal 0.6 l","description":"Deze schaal uit de 'Eleganzia'-reeks is gemaakt uit polycarbonaat, een sterke, harde en transparante kunststof. Door het deksel kan de schaal ook gebruikt worden om voedsel in de koelkast te bewaren. Tupperware, opgericht in 1946 door Earl Tupper, werd beroemd met de massaproductie van kunststof hui …","designer":"Bob Daenen; Vic Cautereels; Vincent Jalet","producer":"Tupperware","design_date":"2002 — 2002","production_date":"ca. 2003 — 2003","design_place":"Aalst","production_place":"","rights_attribution":"In Copyright"},{"inventory_number":"2020-0037_3-4","title":"ronde kom van servies in gebruik tijdens first cla …","description":"","designer":"","producer":"Sabena","design_date":"","production_date":"","design_place":"","production_place":"","rights_attribution":"In Copyright"},{"inventory_number":"2006-0093","title":"Voerbak in keramiek","description":"","designer":"","producer":"Kühn keramik","d

In [58]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search/sample?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&k=100"

[{"inventory_number":"1980-0005","title":"Leerdam Unica LLM 79138","description":"De reeks 'Leerdam Unica' omvat de unieke creaties van Glasfabriek Leerdam naast de serieproductie. Als hoofdontwerper ontwierp Floris Meydam talrijke unica. Deze groene glaskubus is opgebouwd uit op elkaar gelijmde lagen kleurloos vlakglas met groene zijkanten.","designer":"Floris Meydam","producer":"Glasfabriek Leerdam","design_date":"","production_date":"1979 — 1979","design_place":"Leerdam","production_place":"","rights_attribution":"In Copyright"},{"inventory_number":"0484","title":"Sleutelplaat met twee vogels","description":"","designer":"onbekend","producer":"","design_date":"","production_date":"1800 — 1850","design_place":"","production_place":"","rights_attribution":"In Copyright"},{"inventory_number":"0822_18-25_c","title":"Fragment van de voering van een bedbehangsel van h …","description":"Dit classicistisch bed behoorde vermoedelijk toe aan Ferdinand Maria de Lobkowitz, bisschop van Gent van

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search/order/filter?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&k=100"